In [ ]:
import dpkt
import numpy as np

In [ ]:
# 粗粒度

In [ ]:
# 细粒度 (IPD, Packet_Size)
def extract_pcap_s(filename, interval, src, dst):
    
    with open(filename, 'rb') as pcap_file:
        pcap_file = dpkt.pcap.Reader(pcap_file)
        packet_list = []
        for timestamp, buffer in pcap_file:
            eth = dpkt.ethernet.Ethernet(buffer)
            if not isinstance(eth.data, dpkt.ip.IP):
                print ('Non IP Packet type not supported %s\n' % eth.data.__class__.__name__)
                continue
            ip = eth.data
            if ip.src == src and ip.dst == dst:
                packet_list.append({'timestamp' : timestamp, 'size' : len(buffer)})
            elif ip.src == dst and ip.dst == src:
                packet_list.append({'timestamp' : timestamp, 'size' : -1*len(buffer)})
    
    flow = []
    big_pkt = []
    pre_h_time = 0.0
    for i in range(len(packet_list)):
        time_size = []
        time = float(packet_list[i]['timestamp'])
        size = int(packet_list[i]['size'])
        if size > 0:
            ipd = time - pre_h_time
        else:
            ipd = -(time - pre_h_time)
        if float(time) > interval[1]:
            break
        if float(time) < interval[0]:
            continue
        if abs(size) > 512:# ignore ack packets
            if (pre_h_time != 0) and (ipd == 0):
                big_pkt.append(size)
                continue
            if len(big_pkt)!=0:
                last_pkt=flow.pop()
                flow.append({"ipd": last_pkt["ipd"], "size": sum(big_pkt)+last_pkt['size']})
                big_pkt = []
                num_here_big_pkt += 1
            time_size.append (ipd)
            time_size.append(size)
            time_size = np.array (time_size)
            flow.append ({"ipd": time_size[0], "size": time_size[1]})
            pre_h_time = time

    ipd_list = []
    size_list = []
    for packet in flow:
        ipd_list.append(packet['ipd'])
        size_list.append(packet['size'])
    
    return [0.0] + ipd_list[1:] + size_list
